#### ========================================================
#### SEND-TIME OPTIMIZATION: Two-Stage Probabilistic + Harmonic F1
#### Индустриальный стандарт 2025 года (Sber, Tinkoff, Revolut, Nubank)
#### ========================================================

#### 1. ИМПОРТЫ + КОНФИГ

In [1]:
import pandas as pd
import numpy as np
import random
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split
import warnings
from omegaconf import OmegaConf
import datetime
import yaml
warnings.filterwarnings('ignore')

#### 2. FEATURE ENGINEERING ВРЕМЕНИ

In [2]:
def create_time_features(df: pd.DataFrame, ts_column: str = 'send_ts') -> pd.DataFrame:
    """
    Создаёт лучшие на 2025 год признаки времени для STO.
    
    Вход:
        df — DataFrame с колонкой send_ts (datetime)
    
    Почему именно так:
        - sin/cos — единственный способ заставить модель понять цикличность:
          23:00 и 00:00 — рядом, а не на разных концах шкалы
        - Бизнес-флаги (зарплатные дни, выходные) — дают +5–10% к качеству сами по себе
    
    Выход:
        df с новыми колонками: hour_sin, hour_cos, dow_sin, dow_cos, is_weekend, is_payday_zone и т.д.
    """
    print()
    print('Преобразование времени в цикличное:')
    
    df = df.copy()
    ts = pd.to_datetime(df[ts_column])
    
    df['hour'] = ts.dt.hour
    df['dow']  = ts.dt.dayofweek          # 0=понедельник, 6=воскресенье
    df['day']  = ts.dt.day                # 1–31
    
    # === Циклические преобразования (критически важно!) ===
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin']  = np.sin(2 * np.pi * df['dow']  / 7)
    df['dow_cos']  = np.cos(2 * np.pi * df['dow']  / 7)
    df['day_sin']  = np.sin(2 * np.pi * df['day']  / 31)
    df['day_cos']  = np.cos(2 * np.pi * df['day']  / 31)
    
    # === Бизнес-флаги (подстраиваются под РФ/СНГ) ===
    df['is_weekend'] = (df['dow'] >= 5).astype(int)
    
    # Зарплатные дни: с 25-го по 5-е — пик активности
    df['is_payday_zone'] = df['day'].isin(list(range(25,32)) + list(range(1,6))).astype(int)
    
    # Предпраздничные / постпраздничные — можно добавить свои даты
    # df['is_pre_holiday'] = ts.dt.date.isin([...])
    
    # Удобные бакеты
    df['hour_bucket'] = pd.cut(df['hour'], 
                               bins=[0, 6, 10, 17, 20, 24], 
                               labels=['night', 'morning', 'day', 'evening', 'late_evening'],
                               include_lowest=True)
    print('Done')
    return df

#### 3. ОБУЧЕНИЕ МОДЕЛЕЙ

In [3]:
def train_bank_model(X_train, y_train, X_val, y_val, cat_features=[]):
    """
    Модель P(accept_bank = 1 | x, t) — обучается на ВСЕХ данных
    """
    model = CatBoostClassifier(
        iterations=2000,
        depth=10,
        learning_rate=0.03,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        verbose=100,
        early_stopping_rounds=200,
        task_type="GPU" if __import__('torch').cuda.is_available() else "CPU",
        cat_features=cat_features
    )
    
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        use_best_model=True
    )
    return model

def train_user_model(X_train_user, y_train_user, X_val_user, y_val_user, cat_features=[]):
    """
    Модель P(accept_user = 1 | x, t, accept_bank=1) — обучается ТОЛЬКО на bank==1
    """
    model = CatBoostClassifier(
        iterations=1500,
        depth=8,
        learning_rate=0.05,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        verbose=100,
        early_stopping_rounds=200,
        task_type="GPU" if __import__('torch').cuda.is_available() else "CPU",
        cat_features=cat_features
    )
    
    model.fit(
        X_train_user, y_train_user,
        eval_set=(X_val_user, y_val_user),
        use_best_model=True
    )
    return model

#### 4. КАЛИБРОВКА ВЕРОЯТНОСТЕЙ (Isotonic)

In [4]:
def calibrate_model(model, X_calib, y_calib):
    """
    Isotonic Regression — монотонная калибровка, идеально для GBDT.
    Работает лучше Platt/temperature scaling на табличных данных.
    """
    raw_probs = model.predict_proba(X_calib)[:, 1]
    calibrator = IsotonicRegression(out_of_bounds='clip')
    calibrator.fit(raw_probs, y_calib)
    return calibrator

#### 5. ГАРМОНИЧЕСКИЙ F1 КАК СУРРОГАТ

In [5]:
def harmonic_f1(p_bank: float, p_user: float) -> float:
    return 2 * p_bank * p_user / (p_bank + p_user + 1e-12)


# def harmonic_f1(p_bank: float, p_user: float, eps: float = 1e-8) -> float:
#     """
#     Harmonic F1 = 2 / (1/P + 1/R) при P = R = p_bank * p_user
#     Это единственная дифференцируемая и интерпретируемая метрика,
#     которая одновременно максимизирует и банк, и юзер.
#     """
#     p_success = p_bank * p_user
#     if p_success < eps:
#         return 0.0
#     return 2 * p_success / (p_bank + p_user + eps)

#### 6. ПРЕДРАСЧЁТ СЕТКИ ВРЕМЕНИ (168 слотов = 7×24)

In [6]:
def build_time_grid():
    """
    Создаёт датафрейм со всеми возможными часовыми слотами недели.
    Используется и для предрасчёта, и для inference.
    """
    grid = []
    for dow in range(7):
        for hour in range(24):
            grid.append({'dow': dow, 'hour': hour, 'day': 15})  # день не важен для sin/cos
    time_df = pd.DataFrame(grid)
    time_df['send_ts'] = pd.Timestamp('2025-01-01')  # заглушка для create_time_features
    time_df = time_df.astype('string', errors='ignore')
    return time_df

#### 7. ИНФЕРЕНС: выбор лучшего времени для одного клиента

In [8]:
def predict_best_time_for_dataset(
    df: pd.DataFrame,
    time_grid: pd.DataFrame,
    bank_model,
    user_model,
    bank_calibrator,
    user_calibrator,
    cfg,
    top_k: int = 3
) -> pd.DataFrame:
    """
    Для каждого клиента в df ищет топ-top_k лучших времён отправки.
    Возвращает исходный df с новыми колонками.
    """
    # Все фичи из конфига
    cat_features = [f for f in cfg.features.categorical if f in df.columns]
    num_features = [f for f in cfg.features.numeric if f in df.columns]
    time_features = cfg.features.time.all_generated
    feature_cols = [f for f in (cat_features + num_features + time_features) if f in df.columns]

    # Защита: все категориальные — строки + 'missing'
    for col in cat_features:
        if col in df.columns:
            df[col] = df[col].astype('string').fillna('missing')

    results = []

    for idx, client_row in df.iterrows():
        # Дублируем клиента на все 168 слотов
        candidates = pd.DataFrame([client_row.to_dict()] * len(time_grid))

        # Подставляем временные фичи
        for col in time_grid.columns:
            if col in feature_cols:
                candidates[col] = time_grid[col].values

        # Защита от NaN и Categorical
        for col in cat_features:
            if col in candidates.columns:
                candidates[col] = candidates[col].astype('string').fillna('missing')

        X = candidates[feature_cols]

        # Предсказания
        p_bank_raw = bank_model.predict_proba(X)[:, 1]
        p_user_raw = user_model.predict_proba(X)[:, 1]

        p_bank = bank_calibrator.predict(p_bank_raw)
        p_user = user_calibrator.predict(p_user_raw)

        # Harmonic F1 для каждого слота
        scores = np.array([harmonic_f1(pb, pu) for pb, pu in zip(p_bank, p_user)])
        best_idx = np.argsort(scores)[-top_k:][::-1]

        # Собираем топ-k для текущего клиента
        client_results = {}
        for rank, i in enumerate(best_idx, 1):
            slot = time_grid.iloc[i]
            dow = int(slot['dow'])
            hour = int(slot['hour'])
            dow_name = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'][dow]

            client_results.update({
                f'best_time_rank_{rank}_dow': dow,
                f'best_time_rank_{rank}_hour': hour,
                f'best_time_rank_{rank}_dow_name': dow_name,
                f'best_time_rank_{rank}_time_str': f"{dow_name} {hour:02d}:00",
                f'best_time_rank_{rank}_harmonic_f1': round(float(scores[i]), 5),
                f'best_time_rank_{rank}_p_bank': round(float(p_bank[i]), 5),
                f'best_time_rank_{rank}_p_user': round(float(p_user[i]), 5),
            })

        # Добавляем идентификатор клиента (если есть)
        if 'client_id' in client_row:
            client_results['client_id'] = client_row['client_id']

        results.append(client_results)

    result_df = pd.DataFrame(results)

    # Объединяем с исходным df по индексу (или по client_id)
    if 'client_id' in df.columns and 'client_id' in result_df.columns:
        final_df = df[['client_id']].reset_index(drop=True)
        final_df = final_df.merge(result_df, on='client_id', how='left')
    else:
        final_df = pd.concat([df.reset_index(drop=True), result_df], axis=1)

    return final_df

In [9]:
# def predict_best_time(
#     client_row: pd.Series,
#     time_grid: pd.DataFrame,
#     bank_model, user_model,
#     bank_calibrator, user_calibrator,
#     config,                              # ← твой конфиг (OmegaConf)
#     top_k: int = 5
# ):
#     # Все фичи, которые идут в модель
#     all_features = (
#         config.features.categorical +
#         config.features.numeric +
#         config.features.time.all_generated
#     )

#     # Дублируем клиента
#     candidates = pd.DataFrame([client_row.to_dict()] * len(time_grid))

#     # Добавляем временные фичи из time_grid (без дублирования колонок)
#     time_features_to_add = [
#         col for col in config.features.time.all_generated
#         if col in time_grid.columns
#     ]

#     for col in time_features_to_add:
#         candidates[col] = time_grid[col].values

#     # Защита от NaN и Categorical
#     cat_like = config.features.categorical + config.features.time.buckets
#     for col in cat_like:
#         if col in candidates.columns:
#             candidates[col] = candidates[col].astype('string').fillna('missing')

#     # Финальный X
#     X = candidates[all_features]

#     # Предсказания
#     p_bank = bank_calibrator.predict(bank_model.predict_proba(X)[:, 1])
#     p_user = user_calibrator.predict(user_model.predict_proba(X)[:, 1])

#     scores = np.array([harmonic_f1(pb, pu) for pb, pu in zip(p_bank, p_user)])
#     best_idx = np.argsort(scores)[-top_k:][::-1]

#     results = []
#     for i in best_idx:
#         slot = time_grid.iloc[i]
#         results.append({
#             'dow': int(slot['dow']),
#             'hour': int(slot['hour']),
#             'score': round(float(scores[i]), 4),
#             'p_bank': round(float(p_bank[i]), 4),
#             'p_user': round(float(p_user[i]), 4),
#         })
#     return results

#### 8. ПОЛНЫЙ ПАЙПЛАЙН: от данных до модели

In [11]:
def full_pipeline(df_path: str = None):
    config = OmegaConf.load("config.yaml")
    
    if df_path != None:
        # 1. Загрузка
        df = pd.read_parquet(df_path)  # или csv
    else:
        df = generate_data(5000)

    # ← ПЕРЕИМЕНОВЫВАЕМ ТОЛЬКО ТОیار

    df = df.rename(columns={
        config.rename.send_ts: 'send_ts',
        config.rename.accept_bank: 'accept_bank',
        config.rename.accept_user: 'accept_user',
        config.rename.client_id: 'client_id',
    })

    # 2. Feature engineering
    df = create_time_features(df, 'send_ts')
   
    # 3. Определяем признаки
    #cat_features = ['offer_type', 'channel', 'segment', 'hour_bucket', 'country']  #  категориальные
    cat_features = config.features.categorical
    cat_features = [c for c in cat_features if c in df.columns]

    df = df.astype({col: 'string' for col in df.select_dtypes(include=['category', 'object']).columns})
    for col in cat_features:
        df[col] = df[col].fillna('missing').astype(str)

    all_features_from_config = (
        config.features.categorical +
        config.features.numeric +
        config.features.time.all_generated
    )
    
    # Берём только те, что реально есть в датафрейме (защита от опечаток)
    feature_cols = [f for f in all_features_from_config if f in df.columns]
    
    # А cat_features — только те категориальные, что есть
    cat_features = [f for f in config.features.categorical if f in df.columns]

    # 4. Разбиение (temporal!)
    df = df.sort_values('send_ts')
    train_df = df.iloc[:int(0.7*len(df))]
    val_df   = df.iloc[int(0.7*len(df)):int(0.85*len(df))]
    test_df  = df.iloc[int(0.85*len(df)):]
    
    # 5. Bank model — на всех
    bank_model = train_bank_model(
        train_df[feature_cols], train_df['accept_bank'],
        val_df[feature_cols],   val_df['accept_bank'],
        cat_features
    )
    
    # 6. User model — только где bank==1
    train_user = train_df[train_df['accept_bank'] == 1]
    val_user   = val_df[val_df['accept_bank'] == 1]
    
    user_model = train_user_model(
        train_user[feature_cols], train_user['accept_user'],
        val_user[feature_cols],   val_user['accept_user'],
        cat_features
    )
    
    # 7. Калибровка
    bank_calibrator = calibrate_model(bank_model, val_df[feature_cols], val_df['accept_bank'])
    user_calibrator = calibrate_model(user_model, val_user[feature_cols], val_user['accept_user'])
    
    # 8. Сетка времени
    time_grid = build_time_grid()
    
    # 9. Пример предсказания для одного клиента
    # client_example = test_df.iloc[0]
    # best_times = predict_best_time(
    #     client_example, time_grid,
    #     bank_model, user_model,
    #     bank_calibrator, user_calibrator,
    #     config        # ← просто передаём конфиг
    # )
    print("Запуск персонализации для всех клиентов в test_df...")
    personalized_df = predict_best_time_for_dataset(
        df=test_df.copy(),
        time_grid=time_grid,
        bank_model=bank_model,
        user_model=user_model,
        bank_calibrator=bank_calibrator,
        user_calibrator=user_calibrator,
        cfg=config,
        top_k=3
    )

# Сохраняем результат
    personalized_df.to_parquet("personalized_send_times.parquet", index=False)
    print(f"Готово! Сохранено {len(personalized_df)} клиентов с персональным временем")
    print("\nПример:")
    print(personalized_df[[
    'client_id',
    'best_time_rank_1_time_str',
    'best_time_rank_1_harmonic_f1',
    'best_time_rank_1_p_bank',
    'best_time_rank_1_p_user'
        ]].head(10))
    
    print("ТОП-5 лучших времён для клиента:")
    for r in best_times:
        print(f"  День {r['dow']} (пн=0), час {r['hour']:02d}:00 → F1 = {r['score']:.4f} "
              f"(p_bank={r['p_bank']:.3f}, p_user={r['p_user']:.3f})")
    
    return {
        'bank_model': bank_model,
        'user_model': user_model,
        'bank_calibrator': bank_calibrator,
        'user_calibrator': user_calibrator,
        'time_grid': time_grid,
        'feature_cols': feature_cols,
        'cat_features': cat_features
    }

#### 9. ГЕНЕРАЦИЯ ДАННЫХ

In [12]:
# Простой вариант
def generate_data(n = 20000):
    begin = datetime.datetime(2025, 1, 1, 0, 0, 0).timestamp()
    end = datetime.datetime(2025, 11, 27, 14, 30, 0).timestamp()
    
    data = pd.DataFrame({
        'send_ts': np.random.randint(int(begin), int(end), n),
        'country': np.random.choice(['uk', 'us', 'de'], n),
        'age': np.random.randint(18, 65, n),
        'app_intensity': np.random.uniform(0, 10, n),
    })
    
    data['send_ts'] = pd.to_datetime(data['send_ts'], unit='s')
    data['day'] = data['send_ts'].dt.day   
    
    # --- uplift accept_bank на 15-й день для uk ---
    data['y_true'] = ((data['day'] == 15)&(data['country']== 'uk')) * 0.7*np.random.random(len(data)) + 0.3
    data['y_true'] = data['y_true'].clip(0.01, 0.99)
    data['accept_bank'] = np.random.binomial(1, data['y_true'])

    # --- uplift accept_user на 21-й день для us ---
    data['y_true_2'] = ((data['accept_bank'] == 1)&(data['day'] == 21)&(data['country']== 'us')) * 0.3*np.random.random(len(data)) + 0.7
    data['y_true_2'] = data['y_true_2'].clip(0.01, 0.99)
    data['accept_user'] = np.random.binomial(1, data['y_true_2'])
    

    # --- Проверка ---
    print('Сгенерированы данные')
    print(data.sample(3))
    print(data.groupby('day')['accept_bank'].mean()[13:16])
    print(data.groupby('day')['accept_user'].mean()[20:23])

    # --- Убьем лишнее ---
    data = data.drop(columns = ['y_true_2','y_true','day'])
    return data

In [13]:
# # сложный вариант
# n_users = 500000

# users = pd.DataFrame({
#     'user_id': range(n_users),
#     'gender': np.random.choice(['M', 'F'], n_users, p=[0.48, 0.52]),
#     'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'RU'], n_users),
#     'age': np.random.gamma(5, 5, n_users).astype(int).clip(18, 70),
#     'app_intensity': np.random.randint(1, 11, n_users),
#     'device': np.random.choice(['ios', 'android'], n_users, p=[0.4, 0.6]),
#     'has_kids': np.random.choice([0, 1], n_users, p=[0.7, 0.3]),
#     'is_left_handed': np.random.choice([0, 1], n_users, p=[0.9, 0.1]),
#     'eye_color': np.random.choice(['brown', 'blue', 'green'], n_users, p=[0.6, 0.3, 0.1]),
# })

#### ЗАПУСК

In [14]:
if __name__ == "__main__":
    # Замени на свой путь
    models = full_pipeline()
    
    # Сохрани всё для продакшена
    import joblib
    joblib.dump(models, "sto_models_v1.pkl")
    print("Модели сохранены в sto_models_v1.pkl")

Сгенерированы данные
                 send_ts country  age  app_intensity  day  y_true  \
4864 2025-05-15 16:54:57      us   64       8.367233   15     0.3   
4684 2025-05-18 09:59:14      de   43       6.048550   18     0.3   
1819 2025-03-07 11:49:30      de   50       0.334946    7     0.3   

      accept_bank  y_true_2  accept_user  
4864            0       0.7            1  
4684            1       0.7            1  
1819            0       0.7            1  
day
14    0.237838
15    0.394444
16    0.309942
Name: accept_bank, dtype: float64
day
21    0.710983
22    0.693642
23    0.691892
Name: accept_user, dtype: float64

Преобразование времени в цикличное:
Done
0:	test: 0.5153185	best: 0.5153185 (0)	total: 193ms	remaining: 6m 25s
100:	test: 0.5059217	best: 0.5511365 (9)	total: 4.43s	remaining: 1m 23s
200:	test: 0.4972868	best: 0.5511365 (9)	total: 9.2s	remaining: 1m 22s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5511365079
bestIteration = 9

Shrink mod

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.